# =====================================
# 01. Data Cleaning & Merging
# =====================================

**PURPOSE: Clean and merge the 5 raw Telco datasets into a single customer-level table (df_master) with consistent column names and a standard key (customer_id), ready for EDA and modeling.**

### Dataset Summary
| 
| **Demographics**  | 7,043 × 6 | Key: Customer ID  | Demographic attributes (gender, age, marital status). \
| **Location**      | 7,043 × 8 | Key: Customer ID  | Geographic features including country, city, and ZIP code. \
| **Services**      | 7,043 × 11| Key: Customer ID  | Subscribed telecom and streaming services. \
| **Status**        | 7,043 × 9 | Key: Customer ID  | Account details, tenure, churn label, and churn reason. \
| **Population**    | 1,671 × 3 | Key: Zip Code     | ZIP-level population counts; joined later via `Zip Code` from the location table. 

**Merge logic:**
- Customer-level tables (`demographics`, `location`, `services`, `status`) are merged one-to-one on `Customer ID`.  
- `Population` is an auxiliary dataset used for enrichment through `Zip Code`.

## Imports

In [1]:
import sys
from pathlib import Path
import pandas as pd

## Path Configuration 

In [2]:
ROOT = Path.cwd().resolve().parent
sys.path.append(str(ROOT / "src"))

from config import RAW_DIR
from utils_data import save_df, quick_overview

## Load raw tables

In [3]:
FILES = {
    "demographics": "Telco_customer_churn_demographics.xlsx",
    "location":     "Telco_customer_churn_location.xlsx",
    "population":   "Telco_customer_churn_population.xlsx",
    "services":     "Telco_customer_churn_services.xlsx",
    "status":       "Telco_customer_churn_status.xlsx",
}

dfs = {name: pd.read_excel(RAW_DIR / file) for name, file in FILES.items()}

for name, df in dfs.items():
    print(f"{name:12s}: {df.shape[0]} rows × {df.shape[1]} columns")

demographics = dfs["demographics"]
location     = dfs["location"]
population   = dfs["population"]
services     = dfs["services"]
status       = dfs["status"]


demographics: 7043 rows × 9 columns
location    : 7043 rows × 9 columns
population  : 1671 rows × 3 columns
services    : 7043 rows × 30 columns
status      : 7043 rows × 11 columns


## Normalise column names and key 

In [4]:
KEY = "customer_id"

def normalize(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = (
        out.columns
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.lower()
    )
    # Map variations of the customer key
    for cand in ("customerid", "customer_id", "customer id"):
        if cand in out.columns and cand != KEY:
            out = out.rename(columns={cand: KEY})
    return out

demographics = normalize(demographics)
location     = normalize(location)
population   = normalize(population)
services     = normalize(services)
status       = normalize(status)

# Assert key presence only for customer-level tables
for name, df in {
    "demographics": demographics,
    "location":     location,
    "services":     services,
    "status":       status,
}.items():
    assert KEY in df.columns, f"{name} does not contain '{KEY}' after normalization."


## Basic Cleaning and Consistency Checks 

In [5]:
# Ensure unique customer IDs
for name, df in {
    "demographics": demographics,
    "location":     location,
    "services":     services,
    "status":       status,
}.items():
    assert df[KEY].is_unique, f"{name}: duplicate {KEY}s found."

# Handle whitespace or formatting issues if any
for name, df in dfs.items():
    if KEY in df.columns:
        df[KEY] = df[KEY].astype(str).str.strip()


## Prefix Columns and Merge Customer-Level Tables 

In [6]:
def add_prefix_except(df: pd.DataFrame, prefix: str, keep=(KEY,)) -> pd.DataFrame:
    return df.rename(columns={c: (prefix + c) if c not in keep else c for c in df.columns})

demo_ = add_prefix_except(demographics, "demo_")
loc_  = add_prefix_except(location,     "loc_")
svc_  = add_prefix_except(services,     "svc_")
st_   = add_prefix_except(status,       "st_")

df_master = (
    demo_
    .merge(loc_, on=KEY, how="inner", validate="one_to_one")
    .merge(svc_, on=KEY, how="inner", validate="one_to_one")
    .merge(st_,  on=KEY, how="inner", validate="one_to_one")
).set_index(KEY)

print(df_master.shape)
df_master.head()


(7043, 55)


,demo_count,demo_gender,demo_age,demo_under_30,demo_senior_citizen,demo_married,demo_dependents,demo_number_of_dependents,loc_count,loc_country,...,st_count,st_quarter,st_satisfaction_score,st_customer_status,st_churn_label,st_churn_value,st_churn_score,st_cltv,st_churn_category,st_churn_reason
customer_id,,,,,,,,,,,,,,,,,,,,,
8779-QRDMV,1,Male,78,No,Yes,No,No,0,1,United States,...,1,Q3,3,Churned,Yes,1,91,5433,Competitor,Competitor offered more data
7495-OOKFY,1,Female,74,No,Yes,Yes,Yes,1,1,United States,...,1,Q3,3,Churned,Yes,1,69,5302,Competitor,Competitor made better offer
1658-BYGOY,1,Male,71,No,Yes,No,Yes,3,1,United States,...,1,Q3,2,Churned,Yes,1,81,3179,Competitor,Competitor made better offer
4598-XLKNJ,1,Female,78,No,Yes,Yes,Yes,1,1,United States,...,1,Q3,2,Churned,Yes,1,88,5337,Dissatisfaction,Limited range of services
4846-WHAFZ,1,Female,80,No,Yes,Yes,Yes,1,1,United States,...,1,Q3,2,Churned,Yes,1,67,2793,Price,Extra data charges


## Enrich with Zip Code 

In [7]:
if "loc_zip_code" in df_master.columns:
    # Reset index temporarily for merge
    df_master = (
        df_master.reset_index()
        .merge(
            population.rename(columns={
                "zip_code": "loc_zip_code",
                "population": "zip_population"
            })[["loc_zip_code", "zip_population"]],
            on="loc_zip_code",
            how="left"
        )
        .set_index("customer_id")  # restore index
    )


## Data Cleaning Phase 1 – Structural & Integrity Checks
 Verify the integrity of the merged dataset before any transformations:

### Verify dataset shape and unique IDs

In [8]:
print(f"Shape: {df_master.shape}")
print(f"Index name: {df_master.index.name}")

# Check index uniqueness and completeness
assert df_master.index.is_unique, "Duplicate customer_id detected."
assert df_master.index.notna().all(), "Missing customer_id in index."
print("customer_id index is unique and complete.")


Shape: (7043, 56)
Index name: customer_id
customer_id index is unique and complete.


### Remove redundant columns

In [9]:
redundant_cols = ["loc_lat_long", "demo_count", "loc_count", "svc_count", "st_count"]
existing = [c for c in redundant_cols if c in df_master.columns]

if existing:
    df_master.drop(columns=existing, inplace=True)
    print(f"Removed redundant columns: {existing}")
else:
    print("No redundant columns found.")

Removed redundant columns: ['loc_lat_long', 'demo_count', 'loc_count', 'svc_count', 'st_count']


### Remove constant columns

In [10]:
constant_cols = df_master.columns[df_master.nunique() <= 1].tolist()

if constant_cols:
    print(f"⚠️ Constant columns detected: {constant_cols}")
else:
    print("✅ No constant columns found.")

⚠️ Constant columns detected: ['loc_country', 'loc_state', 'svc_quarter', 'st_quarter']


In [11]:
if constant_cols:
    df_master.drop(columns=constant_cols, inplace=True)

print(f"✅ Phase 1 complete - new dataset shape: {df_master.shape}")

✅ Phase 1 complete - new dataset shape: (7043, 47)


## Data Cleaning Phase 2 – Data Type & Value Normalization
Make sure each column is in the correct format for EDA and modeling.

### Inspect current dtypes

In [12]:
dtype_summary = (
    df_master.dtypes
    .reset_index()
    .rename(columns={'index': 'column', 0: 'dtype'})
    .sort_values('dtype')
)
display(dtype_summary)

,column,dtype
46,zip_population,int64
20,svc_avg_monthly_gb_download,int64
35,svc_total_extra_data_charges,int64
38,st_satisfaction_score,int64
12,svc_number_of_referrals,int64
8,loc_zip_code,int64
13,svc_tenure_in_months,int64
41,st_churn_value,int64
42,st_churn_score,int64
43,st_cltv,int64


### Numeric Columns (19 total)
- zip_population
    - Population by ZIP code.
    - ✅ Numeric. Optional: scale later for modeling.
- svc_avg_monthly_gb_download
    - Internet data usage
    - ✅ Numeric
    - 💡 May contain outliers - check distribution.
- svc_total_extra_data_charges
    - Charges due to data overages
    - ✅ Numeric
    - 💡 Inspect for zeros - potential imbalance.
- svc_number_of_referrals
    - Number of friends referred
    - ✅ Numeric
    - 💡 Likely skewed - may be zero for most customers.
- loc_zip_code
    - ⚠️ ZIP codes are identifiers, not numeric - convert to str to avoid losing leading 0
- st_satisfaction_score
    - Customer satisfaction rating (1–5).
    - ✅ Numeric
    - 💡 Check correlation with churn
- demo_number_of_dependents
    - Number of dependents
    - ✅ Numeric
- svc_tenure_in_months
    - Time as customer
    - ✅ Numeric
    - 💡 Might be key churn driver - possible binning for interpretability
- st_churn_score
    - Internal churn risk (0–100)	
    - ⚠️ Potential target leakage — may be derived from or highly correlated with the actual churn label
- st_cltv
    - Customer lifetime value	
    - ✅ Numeric
    - 💡 Scale or log-transform if very skewed
- st_churn_value
    - Churn indicator (0,1) 
    - ⚠️ Redundant with st_churn_label - keep only one.
- demo_age	
    - ✅ Numeric
    - 💡 scale or bin
- svc_avg_monthly_long_distance_charges	
    - ✅ Numeric
- loc_latitude, loc_longitude	
    - ✅ Numeric
    - 💡 Drop if not doing geospatial analysis; or keep for clustering/region mapping.
- svc_total_revenue, svc_total_long_distance_charges, svc_total_refunds, svc_total_charges, svc_monthly_charge
    - ✅ Numeric
    - Financial metrics	
    - 💡 High correlation expected - may later perform feature selection to reduce redundancy.


#### Converting **loc_zip_code** to str

In [13]:
df_master["loc_zip_code"] = df_master["loc_zip_code"].astype(str).str.zfill(5)
df_master["loc_zip_code"].dtype


dtype('O')

### Categorical Columns (27 total)
- demo_gender	
    - Male/Female
    - ✅ Categorical
    - 💡 Keep as is for clarity in EDA; encode as (0,1) for modeling.
- demo_under_30, demo_senior_citizen, demo_married, demo_dependents	
    - Yes/No flags
    - ✅ Categorical
    - 💡 Keep as is for clarity in EDA and encode as (0,1) for modeling
    - ⚠️ demo_dependents might be redundant with demo_number_of_dependents
    - ⚠️ age related variables might be redundant with age
- loc_city	
    - ⚠️ High cardinality, might not generalize well in modelling -  group by region or drop.
- loc_zip_code	ZIP code	
    - ✅ Categorical
    - ZIP code (identifier)
    - 💡 Possible join with region-level features (already merged zip_population). Not for modeling directly.
- svc_referred_a_friend	
    - ✅ Categorical
    - Yes/No	
    - 💡 Keep as is for clarity in EDA and encode as (0,1) for modeling
- svc_offer	
    - Offer name or plan
    - ✅ Categorical
    - 💡 Use one-hot or frequency encoding
- svc_multiple_lines	
    - Indicates if the customer has multiple phone lines (e.g. family members)
    - Yes/No
    - ⚠️ inspect
    - 💡 Keep as is for clarity in EDA and encode as (0,1) for modeling
- svc_internet_service, svc_internet_type	
    - Type of internet connection	
    - ✅ Categorical
    - Possibly hierarchical — one-hot encode.
- svc_online_security, svc_online_backup, svc_device_protection_plan, svc_premium_tech_support, svc_streaming_tv, svc_streaming_movies, svc_streaming_music, svc_unlimited_data	
    - ✅ Categorical.
    - 💡 Normalize “No internet” vs. “No” for consistency, then encode as binary (1/0) or ordinal.
- svc_contract	
    - Contract type (Month-to-month, One year, Two year)	
    - ✅ Ordered categorical.
    - 💡 Encode as ordinal (0/1/2).
- svc_paperless_billing	
    - ✅ Categorical.
    - 💡 Keep as is for clarity in EDA; encode as (0,1) for modeling.
- svc_payment_method	
    - ✅ Categorical.
    - 💡 One-hot encode.
- st_customer_status	
    - Active/Churned/Joined	
    - ⚠️ Potential leakage — remove for modeling. Keep for churn diagnostics only.
- st_churn_label	
    - ✅ Binary target.
    - 💡 Encode as (1 if “Yes”, 0 if “No”).
- st_churn_category, st_churn_reason	
    - Reason for churn	
    - ⚠️ Leakage — only known post-churn. Drop before modeling.



#### Verifying Consistency of All Binary Columns 

In [14]:
binary_cats = [
    "demo_under_30", "demo_senior_citizen", "demo_married", "demo_dependents",
    "svc_referred_a_friend", "svc_multiple_lines",
    "svc_online_security", "svc_online_backup", "svc_device_protection_plan",
    "svc_premium_tech_support", "svc_streaming_tv", "svc_streaming_movies",
    "svc_streaming_music", "svc_unlimited_data", "svc_paperless_billing"
]

expected = {"Yes", "No"}
for col in binary_cats:
    diff = set(df_master[col].dropna().unique()) - expected
    if diff:
        print(f"{col}: unexpected values → {diff}")


# No unexpected values found

In [15]:
df_master["svc_multiple_lines"].value_counts(dropna=False)


svc_multiple_lines
No     4072
Yes    2971
Name: count, dtype: int64

## Data Cleaning Phase 3 – Handling Missing Values


In [16]:
missing = df_master.isna().sum().sort_values(ascending=False)
missing = missing[missing > 0]

if not missing.empty:
    print("Columns with missing values:")
    display(missing)
else:
    print("No missing values found in df_master.")

Columns with missing values:


st_churn_reason      5174
st_churn_category    5174
svc_offer            3877
svc_internet_type    1526
dtype: int64

### Filling Missing Values

In [17]:
# Missing churn details (st_churn_reason & st_churn_category) is meaningful = customer didn’t churn

# Missing value summary of the before for clarity
missing_before = df_master.isna().sum()

# Handling missing values
df_master["svc_offer"] = df_master["svc_offer"].fillna("No offer")
df_master["svc_internet_type"] = df_master["svc_internet_type"].fillna("No internet service")

# Missing value summary table
missing_after = df_master.isna().sum()
missing_summary = pd.DataFrame({
    "before": missing_before,
    "after": missing_after
}).query("before > 0 or after > 0")

display(missing_summary)


,before,after
svc_offer,3877,0
svc_internet_type,1526,0
st_churn_category,5174,5174
st_churn_reason,5174,5174


## Data Cleaning Phase 4 – Target Validation
Inspecting churn related variables and ensure no label leakage

#### Inspecting the potential leakage columns:
`st_customer_status`, `st_churn_category`, `st_churn_reason`

In [18]:
leakage_candidates = [
    "st_customer_status",
    "st_churn_category",
    "st_churn_reason",
]

target_col = "st_churn_value"

# Inspect distribution and relationship with churn
for col in leakage_candidates:
    if col not in df_master.columns:
        continue
    
    print(f"\n===== {col} =====")
    print(df_master[col].value_counts(dropna=False).head(15))
    
    churn_by_col = (
        df_master.groupby(col)[target_col]
        .agg(["mean", "count"])
        .sort_values("mean", ascending=False)
    )
    print("\nChurn rate by category:")
    display(churn_by_col.head(15))



===== st_customer_status =====
st_customer_status
Stayed     4720
Churned    1869
Joined      454
Name: count, dtype: int64

Churn rate by category:


,mean,count
st_customer_status,,
Churned,1.0,1869
Joined,0.0,454
Stayed,0.0,4720



===== st_churn_category =====
st_churn_category
NaN                5174
Competitor          841
Attitude            314
Dissatisfaction     303
Price               211
Other               200
Name: count, dtype: int64

Churn rate by category:


,mean,count
st_churn_category,,
Attitude,1.0,314
Competitor,1.0,841
Dissatisfaction,1.0,303
Other,1.0,200
Price,1.0,211



===== st_churn_reason =====
st_churn_reason
NaN                                          5174
Competitor had better devices                 313
Competitor made better offer                  311
Attitude of support person                    220
Don't know                                    130
Competitor offered more data                  117
Competitor offered higher download speeds     100
Attitude of service provider                   94
Price too high                                 78
Product dissatisfaction                        77
Network reliability                            72
Long distance charges                          64
Service dissatisfaction                        63
Moved                                          46
Extra data charges                             39
Name: count, dtype: int64

Churn rate by category:


,mean,count
st_churn_reason,,
Attitude of service provider,1.0,94
Attitude of support person,1.0,220
Competitor had better devices,1.0,313
Competitor made better offer,1.0,311
Competitor offered higher download speeds,1.0,100
Competitor offered more data,1.0,117
Deceased,1.0,6
Don't know,1.0,130
Extra data charges,1.0,39


#### Drop Confirmed Leakage Columns
All 3 columns contain post-event information directly derived from churn outcomes.
Keeping them would cause severe target leakage and inflated model performance.

In [19]:
df_master.drop(columns=leakage_candidates, inplace=True)

print("✅ Dropped confirmed leakage columns:", leakage_candidates)

✅ Dropped confirmed leakage columns: ['st_customer_status', 'st_churn_category', 'st_churn_reason']


#### Rename churn column for modeling
The column 'st_churn_value' is our final churn indicator (1 = churned, 0 = not churned).
Renaming it to 'churn_flag' for clarity.

In [20]:
df_master.rename(columns={"st_churn_value": "churn_flag"}, inplace=True)

print("✅ Renamed 'st_churn_value' to 'churn_flag' (1 = churned, 0 = not churned).")

✅ Renamed 'st_churn_value' to 'churn_flag' (1 = churned, 0 = not churned).


## Data Cleaning Phase 5 – Dataset Housekeeping


In [21]:
# === Final dataset save and summary ===

# Save cleaned dataset
final_path = save_df(df_master, "cleaned_df", folder="interim", fmt="csv")

# Identify feature types
num_cols = df_master.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = df_master.select_dtypes(include=["object"]).columns.tolist()

print(f"🧮 Shape:             {df_master.shape[0]:,} rows × {df_master.shape[1]} columns")
print(f"🔢 Numeric features:  {len(num_cols)}")
print(f"🔠 Categorical:       {len(cat_cols)}")
print("✅ Dataset ready for EDA and modeling.")


✅ DataFrame saved to: /Users/pedroazevedo/Documents/GitHub/EnterpriseDataScienceBootcamp_workgroup/data/interim/cleaned_df.csv
🧮 Shape:             7,043 rows × 44 columns
🔢 Numeric features:  19
🔠 Categorical:       25
✅ Dataset ready for EDA and modeling.
